In [1]:
import warnings

warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score,recall_score, accuracy_score, f1_score, roc_auc_score
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

# Classification
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

training_data = 'adult.data.csv'
test_data = 'adult.test.csv'
columns = ['Age', 'Workclass', 'Final_Weight', 'Education', 'Education Num', 'Marital Status',
           'Occupation', 'Relationship', 'Race', 'Sex', 'Capital Gain', 'Capital Loss',
           'Hours_Per_Week', 'Country', 'Income_Class']
train = pd.read_csv(training_data, names=columns, delimiter=' *, *')
test = pd.read_csv(test_data, names=columns, delimiter=' *, *')

test = test.iloc[1:]


# seeing where the na values are
print(train.isin(["?"]).sum())
print("----------------------------------------")
print(test.isin(["?"]).sum())



Age                  0
Workclass         1836
Final_Weight         0
Education            0
Education Num        0
Marital Status       0
Occupation        1843
Relationship         0
Race                 0
Sex                  0
Capital Gain         0
Capital Loss         0
Hours_Per_Week       0
Country            583
Income_Class         0
dtype: int64
----------------------------------------
Age                 0
Workclass         963
Final_Weight        0
Education           0
Education Num       0
Marital Status      0
Occupation        966
Relationship        0
Race                0
Sex                 0
Capital Gain        0
Capital Loss        0
Hours_Per_Week      0
Country           274
Income_Class        0
dtype: int64


In [2]:
train = train.replace('?', np.nan)

train.dropna(how='any', inplace=True)

test = test.replace('?', np.nan)
test.dropna(how='any', inplace=True)

# getting rid of the full stops
test.head()
test = test.replace("<=50K.", "<=50K")
test = test.replace(">50K.", ">50K")
test.head()

# dropping education as there is also a numerical column for this
train = train.drop('Education', 1)
test = test.drop('Education', 1)

print(train.head(2))
print("------------------------------------------")
print(train.head(2))

   Age         Workclass  Final_Weight  Education Num      Marital Status  \
0   39         State-gov         77516             13       Never-married   
1   50  Self-emp-not-inc         83311             13  Married-civ-spouse   

        Occupation   Relationship   Race   Sex  Capital Gain  Capital Loss  \
0     Adm-clerical  Not-in-family  White  Male          2174             0   
1  Exec-managerial        Husband  White  Male             0             0   

   Hours_Per_Week        Country Income_Class  
0              40  United-States        <=50K  
1              13  United-States        <=50K  
------------------------------------------
   Age         Workclass  Final_Weight  Education Num      Marital Status  \
0   39         State-gov         77516             13       Never-married   
1   50  Self-emp-not-inc         83311             13  Married-civ-spouse   

        Occupation   Relationship   Race   Sex  Capital Gain  Capital Loss  \
0     Adm-clerical  Not-in-family  W

In [3]:
print(train.isnull().sum()) 
print("=========================================")
print(test.isnull().sum()) 

Age               0
Workclass         0
Final_Weight      0
Education Num     0
Marital Status    0
Occupation        0
Relationship      0
Race              0
Sex               0
Capital Gain      0
Capital Loss      0
Hours_Per_Week    0
Country           0
Income_Class      0
dtype: int64
Age               0
Workclass         0
Final_Weight      0
Education Num     0
Marital Status    0
Occupation        0
Relationship      0
Race              0
Sex               0
Capital Gain      0
Capital Loss      0
Hours_Per_Week    0
Country           0
Income_Class      0
dtype: int64


In [4]:

# creating the dummie variables for train which will change categorical attribute to numeric
Categorical_variables = ['Workclass', 'Marital Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country']

for var in Categorical_variables:
    Categorical_list = 'var' + '_' + var
    Categorical_list = pd.get_dummies(train[var], prefix=var)
    data1 = train.join(Categorical_list)
    train = data1

Categorical_variables = ['Workclass', 'Marital Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country']
data_variables = train.columns.values.tolist()
to_keep = [i for i in data_variables if i not in Categorical_variables]

train = train[to_keep]
train['Income_Class'] = LabelEncoder().fit_transform(train['Income_Class'])
# if('Income_Class' == ' >50K' or 'Income_Class' == ' >50K'):return 1
# if('Income_Class' == ' <=50K' or 'Income_Class' == ' <=50K'):return 0
# train['Income_Class']   = train.apply(lambda row: binarize(row['Income_Class']) , axis = 1)
train.columns.values

# creating the dummie variables for test which will change categorical attribute to numeric
Categorical_variables = ['Workclass', 'Marital Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country']

for var in Categorical_variables:
    Categorical_list = 'var' + '_' + var
    Categorical_list = pd.get_dummies(test[var], prefix=var)
    data1 = test.join(Categorical_list)
    test = data1

Categorical_variables = ['Workclass', 'Marital Status', 'Occupation', 'Relationship', 'Race', 'Sex', 'Country']
data_variables = test.columns.values.tolist()
to_keep = [i for i in data_variables if i not in Categorical_variables]

test = test[to_keep]
test['Income_Class'] = LabelEncoder().fit_transform(test['Income_Class'])

print(test.head(2))

   Age  Final_Weight  Education Num  Capital Gain  Capital Loss  \
1   38         89814              9             0             0   
2   28        336951             12             0             0   

   Hours_Per_Week  Income_Class  Workclass_Federal-gov  Workclass_Local-gov  \
1              50             0                      0                    0   
2              40             1                      0                    1   

   Workclass_Private  ...  Country_Portugal  Country_Puerto-Rico  \
1                  1  ...                 0                    0   
2                  0  ...                 0                    0   

   Country_Scotland  Country_South  Country_Taiwan  Country_Thailand  \
1                 0              0               0                 0   
2                 0              0               0                 0   

   Country_Trinadad&Tobago  Country_United-States  Country_Vietnam  \
1                        0                      1                0  

In [5]:
# Check balance
print(train['Income_Class'].value_counts()[0] / train.shape[0])
print(train['Income_Class'].value_counts()[1] / train.shape[0])


0.7510775147536636
0.24892248524633645


In [6]:


print(train['Income_Class'])



0        0
1        0
2        0
3        0
4        0
        ..
32556    0
32557    1
32558    0
32559    0
32560    1
Name: Income_Class, Length: 30162, dtype: int64


In [7]:
print(train.isnull().sum()) # Check missing values
print("=========================================")
print(test.isnull().sum()) # Check missing values

Age                        0
Final_Weight               0
Education Num              0
Capital Gain               0
Capital Loss               0
                          ..
Country_Thailand           0
Country_Trinadad&Tobago    0
Country_United-States      0
Country_Vietnam            0
Country_Yugoslavia         0
Length: 89, dtype: int64
Age                        0
Final_Weight               0
Education Num              0
Capital Gain               0
Capital Loss               0
                          ..
Country_Thailand           0
Country_Trinadad&Tobago    0
Country_United-States      0
Country_Vietnam            0
Country_Yugoslavia         0
Length: 88, dtype: int64


In [8]:
# seeing if they have the same number of columns
print(test.shape)
print(train.shape)
# they don't so need to find out what that is



(15059, 88)
(30162, 89)


In [9]:
# checking to see which column is missing
missing_cols = set(train) - set(test)
print(missing_cols)



{'Country_Holand-Netherlands'}


In [10]:
# Adding in a column that was missing from the test set filled with 0's
test['Country_Holand-Netherlands'] = pd.Series(0, index=test.index)

# Standardize the inputs
scaler = StandardScaler()
scaler.fit(train)
train_data = scaler.transform(train)
test_data = scaler.transform(test)

# making the training and test set
X_train = train.drop(['Income_Class'], axis=1)
y_train = train['Income_Class']
X_test = test.drop(['Income_Class'], axis=1)
y_test = test['Income_Class']

def showResult(classifier, label):
    start_time = datetime.datetime.now()
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    end_time = datetime.datetime.now()  # Track learning ending time
    exection_time = (end_time - start_time).total_seconds()  # Track execution time
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    roc_AUC = roc_auc_score(y_test, y_pred)

    print("Classifier: " + label)
    print(label + ' Accuracy : %0.2f ' % acc)
    print(label + ' Precision Score : %0.2f ' % prec)
    print(label + ' F1 Score : %0.2f ' % F1)
    print(label + ' Recall Score : %0.2f ' % rec)
    print(label + ' roc Score: %0.2f ' % roc_AUC)
    print("Learn: execution time={t:.3f} seconds".format(t=exection_time))
    print("--------------------------")


showResult(KNeighborsClassifier(n_neighbors=15, weights='distance'), "KNN")
showResult(GaussianNB(), "Naive Bayes")
showResult(SVC(), "SVM")
showResult(DecisionTreeClassifier(max_depth=10), "Decision Tree")
showResult(RandomForestClassifier(), "Random Forest")
showResult(AdaBoostClassifier(), "AdaBoost")
showResult(GradientBoostingClassifier(), "Gradient Boosting")
showResult(LinearDiscriminantAnalysis(), "Linear Discriminant")
showResult(MLPClassifier(), "MLP")
showResult(LogisticRegression(penalty='l1'), "Logistic Regression")

Classifier: KNN
KNN Accuracy : 0.79 
KNN Precision Score : 0.64 
KNN F1 Score : 0.42 
KNN Recall Score : 0.31 
KNN roc Score: 0.63 
Learn: execution time=1.563 seconds
--------------------------
Classifier: Naive Bayes
Naive Bayes Accuracy : 0.79 
Naive Bayes Precision Score : 0.65 
Naive Bayes F1 Score : 0.42 
Naive Bayes Recall Score : 0.31 
Naive Bayes roc Score: 0.63 
Learn: execution time=0.108 seconds
--------------------------
Classifier: SVM
SVM Accuracy : 0.75 
SVM Precision Score : 0.46 
SVM F1 Score : 0.10 
SVM Recall Score : 0.06 
SVM roc Score: 0.52 
Learn: execution time=423.792 seconds
--------------------------
Classifier: Decision Tree
Decision Tree Accuracy : 0.84 
Decision Tree Precision Score : 0.70 
Decision Tree F1 Score : 0.65 
Decision Tree Recall Score : 0.61 
Decision Tree roc Score: 0.76 
Learn: execution time=0.228 seconds
--------------------------
Classifier: Random Forest
Random Forest Accuracy : 0.85 
Random Forest Precision Score : 0.77 
Random Forest F